# `NLopt` with autodiff and numerical gradients
Presented by Chiyoung Ahn (https://github.com/chiyahn)

This notebook demonstrates how nonlinear optimization problems in `NLopt` can be solved without the need of specifying analytic formulae for gradients by autodifferentiation or numerical gradients.

In [ ]:
] add NLopt BenchmarkTools ForwardDiff

In [3]:
using NLopt, BenchmarkTools, ForwardDiff

## Using `NLopt`

### Nonlinear optimization without nonlinear constraints:

First, define the objective function `f` and the corresponding gradient `g!`. In `NLopt`, evaluation of `f` and execution of `g!` take place in the same time:

In [4]:
# call g! then return f(x)
function fg!(x::Vector, grad::Vector)
    if length(grad) > 0 # gradient of f(x)
        grad[1] = -2*x[1]*(x[1]^2 + x[2]^2)
        grad[2] = -2*x[2]*(x[1]^2 + x[2]^2)
    end
    return -(x[1]^2 + x[2]^2)
end

fg! (generic function with 1 method)

and the corresponding optimization problem:

In [5]:
opt = Opt(:LD_LBFGS, 2) # 2 indicates the length of `x`
lower_bounds!(opt, [-1.0, -1.0]) # find `x` above -2.0
upper_bounds!(opt, [2.0, 2.0]) # find `x` below 2.0
min_objective!(opt, fg!) # specifies that optimization problem is on minimization

and solve it!

In [6]:
(minf,minx,ret) = @btime optimize(opt, [1.0, 1.0])
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

  3.286 ms (21 allocations: 976 bytes)
got -8.0 at [2.0, 2.0] after 2 iterations (returned SUCCESS)


### Nonlinear optimization with nonlinear constraints:

Define `fg!` first:

In [7]:
function fg!(x::Vector, grad::Vector)
    if length(grad) > 0 # gradient of f(x)
        grad[1] = 0
        grad[2] = 0.5/sqrt(x[2])
    end
    return sqrt(x[2]) # f(x)
end

fg! (generic function with 1 method)

and the corresponding optimization problem:

In [8]:
opt = Opt(:LD_SLSQP, 2) # 2 indicates the length of `x`
lower_bounds!(opt, [-Inf, 0.]) # forces `x` to have a non-negative value
min_objective!(opt, fg!) # specifies that optimization problem is on minimization
xtol_rel!(opt,1e-4) # set a lower relative xtol for convergence criteria

Similarly for constraint, where `constraint_f(x) <= 0` is imposed for all `x` 

In [9]:
function constraint_fg!(x::Vector, grad::Vector, a, b)
    if length(grad) > 0 # gradient of constraint_f(x)
        grad[1] = 3a * (a*x[1] + b)^2
        grad[2] = -1
    end
    (a*x[1] + b)^3 - x[2] # constraint_f(x); constraint_f(x) <= 0 is imposed
end

constraint_fg! (generic function with 1 method)

Here, `a` and `b` are added to allow variants of `constraint_f(x)` in a handy way. For instance, to impose
```julia
(2*x[1] + 0)^3 - x[2] <= 0
```
AND
```julia
(-1*x[1] + 1)^3 - x[2] <= 0
```
one can simply run the following two lines:

In [10]:
inequality_constraint!(opt, (x,g) -> constraint_fg!(x,g,2,0), 1e-8)
inequality_constraint!(opt, (x,g) -> constraint_fg!(x,g,-1,1), 1e-8)

Ready to roll out:

In [11]:
(minf,minx,ret) = @btime optimize(opt, [1.234, 5.678])
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

  14.115 μs (242 allocations: 9.03 KiB)
got 0.5443310539518157 at [0.333333, 0.296296] after 13 iterations (returned XTOL_REACHED)


## Using `NLopt` without analytic formulae for gradients

Suppose that `f` you want to optimize is written in a fairly complex form and you do not have an access to the analytic formula of the gradient of `f`. Here are some possible solutions:

### Numerical gradients

First, define `f` from the problem without nonlinear constraints above:

In [17]:
function f(x::Vector)
    return -(x[1]^2 + x[2]^2)
end

f (generic function with 1 method)

Then we can define the corresponding numerical derivative `g!` and the `fg!` using forward-difference with some predetermined `EPS`:

In [18]:
# compute gradient by forward-difference with EPS for epsilon
function g!(G::Vector, x::Vector; EPS = 1e-8)
    EPS = 1e-8
    for i in 1:length(G)
        x_forward = copy(x)
        x_forward[i] += EPS
        G[i] = (f(x_forward) - f(x)) / EPS
    end
end

function fg!(x::Vector, grad::Vector)
    if length(grad) > 0 # gradient of f(x)
        g!(grad, x)
    end
    f(x)
end

fg! (generic function with 1 method)

Solve:

In [19]:
# define the optimization problem
opt = Opt(:LD_LBFGS, 2) # 2 indicates the length of `x`
lower_bounds!(opt, [-1.0, -1.0]) # find `x` above -2.0
upper_bounds!(opt, [2.0, 2.0]) # find `x` below 2.0
min_objective!(opt, fg!) # specifies that optimization problem is on minimization

# solve the optimization problem
(minf,minx,ret) = @btime optimize(opt, [1.0, 1.0])
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

  3.284 ms (24 allocations: 1.31 KiB)
got -8.0 at [2.0, 2.0] after 2 iterations (returned SUCCESS)


### Automatic differentiation

Or using automatic differentiation:

In [20]:
# compute gradient by forward automatic differentiation
function g!(G::Vector, x::Vector)
    ForwardDiff.gradient!(G, f, x)
end

function fg!(x::Vector, grad::Vector)
    if length(grad) > 0 # gradient of f(x)
        g!(grad, x)
    end
    f(x)
end

fg! (generic function with 1 method)

Solve:

In [21]:
# define the optimization problem
opt = Opt(:LD_LBFGS, 2) # 2 indicates the length of `x`
lower_bounds!(opt, [-1.0, -1.0]) # find `x` above -2.0
upper_bounds!(opt, [2.0, 2.0]) # find `x` below 2.0
min_objective!(opt, fg!) # specifies that optimization problem is on minimization

# solve the optimization problem
(minf,minx,ret) = @btime optimize(opt, [1.0, 1.0])
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

  3.294 ms (24 allocations: 1.28 KiB)
got -8.0 at [2.0, 2.0] after 2 iterations (returned SUCCESS)


## Integration with `NLSolversBase`

(TODO)